In [1]:
import cv2
import numpy as np

#Calculating Gaussian pyramid part
#For image1, image2 and mask
def gaussianPyramid(img, pyramidLevel):
    #I defined the first element of the Gaussian pyramid to be created as the picture itself.
    gaussian = [img]
    #I reduced the size of the image using pyrDown
    resizedImg = cv2.pyrDown(img)
    
    #I created loop as many as the specified pyramid level
    for i in range(pyramidLevel):
        #I added the reduced image to the Gaussian pyramid
        gaussian.append(np.float32(resizedImg))
        #I reduced the size of the image using pyrDown again
        resizedImg = cv2.pyrDown(resizedImg)
    
    #It returns gaussian final pyramid
    return gaussian


#Calculating Laplacian pyramid part
#For image1, image2
def laplacianPyramid(gaussianIm):
    
    #I set the top point of Laplacian pyramid as the last element of the gaussianIm array.
    laplacian=[gaussianIm[-1]] 
    #I created a loop starting from end of the array.
    for i in range((len(gaussianIm) - 1),0,-1):
        #Each loop round enlarges the Gaussian pyramid in the ith element
        enlargedGaussian=cv2.pyrUp(gaussianIm[i]) 
        #I subtracted magnified pyramid from the pyramid at element i-1
        subLaplacian=np.subtract(gaussianIm[i-1], enlargedGaussian) 
        # I added all result to the laplacian pyramid
        laplacian.append(subLaplacian)
    #It returns laplacian final pyramid
    return laplacian 


#Finding blended two photos with their mask part
def blending(laplacianIm1, laplacianIm2, gaussianMask):
    blendedIm = []
    size=len(laplacianIm1)
    for i in range(size):
        #I applied Li12 = Li1.Ri + Li2.(1 − Ri) formula below
        element = laplacianIm2[i] * gaussianMask[i] + laplacianIm1[i] * (1.0 - gaussianMask[i])
        #I added each formed element to the blendedIm array
        blendedIm.append(element)
    return blendedIm

#Reconstructing Image part
def reconstruct(blendedIm):
    laplacian=[blendedIm[0]]
    for i in range(len(blendedIm) - 1):
        enlargedLaplacian=cv2.pyrUp(blendedIm[0])
        blendedIm[0]=cv2.add(blendedIm[i+1],enlargedLaplacian)
        laplacian.append(blendedIm[0])
    return laplacian

# Loading the two images for blending
img1=cv2.imread('../data/Input5-1.jpg')
img2=cv2.imread('../data/Input5-2.jpg')

#Creating different masks for each input

#I took the dimensions of the img1 image and created an array named mask1
#And I gave all its elements a value of 0
mask1 = np.zeros(img1.shape[:2], dtype="uint8")
#I drew a rectangle inside this array at the intervals 
#I specified and made the inside of this rectangle white
cv2.rectangle(mask1, (197, 140), (347, 180), 255, -1)
#I saved the mask1 image I created as a jpg with the name Mask1.jpg to use it in the appropriate inputs.
cv2.imwrite('../data/Mask1.jpg',mask1)
mask1=cv2.imread('../data/Mask1.jpg').astype(np.float32)
#I limited all elements of the array from 0 to 1.
mask1=np.clip(mask1, 0, 1)

mask2 = np.zeros(img1.shape[:2], dtype="uint8")
#I drew a rectangle inside this array at the intervals 
#I specified and made the inside of this rectangle white
cv2.rectangle(mask2, (256, 0), (512, 512), 255, -1)
#I saved the mask2 image I created as a jpg with the name Mask2.jpg to use it in the appropriate inputs.
cv2.imwrite('../data/Mask2.jpg',mask2)
mask2=cv2.imread('../data/Mask2.jpg').astype(np.float32)
#I limited all elements of the array from 0 to 1.
mask2=np.clip(mask2, 0, 1)

mask3 = np.zeros(img1.shape[:2], dtype="uint8")
#I drew a rectangle inside this array at the intervals 
#I specified and made the inside of this rectangle white
cv2.rectangle(mask3, (216, 144), (394, 360), 255, -1)
#I saved the mask3 image I created as a jpg with the name Mask3.jpg to use it in the appropriate inputs.
cv2.imwrite('../data/Mask3.jpg',mask3)
mask3=cv2.imread('../data/Mask3.jpg').astype(np.float32)
#I limited all elements of the array from 0 to 1.
mask3=np.clip(mask3, 0, 1)

mask4 = np.zeros(img1.shape[:2], dtype="uint8")
#I drew a rectangle inside this array at the intervals 
#I specified and made the inside of this rectangle white
cv2.rectangle(mask4, (0, 0), (512, 256), 255, -1)
#I saved the mask4 image I created as a jpg with the name Mask4.jpg to use it in the appropriate inputs.
cv2.imwrite('../data/Mask4.jpg',mask4)
mask4=cv2.imread('../data/Mask4.jpg').astype(np.float32)
#I limited all elements of the array from 0 to 1.
mask4=np.clip(mask4, 0, 1)

mask5 = np.zeros(img1.shape[:2], dtype="uint8")
#I drew an ellipse inside this array at the intervals 
#I specified and made the inside of this ellipse white
cv2.ellipse(mask5, (270, 160), (100,85), 90, 0, 360, 255, -1)
#I saved the mask5 image I created as a jpg with the name Mask5.jpg to use it in the appropriate inputs.
cv2.imwrite('../data/Mask5.jpg',mask5)
mask5=cv2.imread('../data/Mask5.jpg').astype(np.float32)
#I limited all elements of the array from 0 to 1.
mask5=np.clip(mask5, 0, 1)


#I defined this value as 7, which is the number that usually produces the best results.
pyramidLevel = 7

# For calculate Gaussian parts
gaussianIm1=gaussianPyramid(img1, pyramidLevel)
gaussianIm2=gaussianPyramid(img2, pyramidLevel)

gaussianMask=gaussianPyramid(mask5, pyramidLevel)
gaussianMask.reverse()

# For calculate Laplacian parts
laplacianIm1=laplacianPyramid(gaussianIm1)
laplacianIm2=laplacianPyramid(gaussianIm2)

# Blend the images
blendedIm=blending(laplacianIm1,laplacianIm2,gaussianMask)
# Reconstruct the images
result=reconstruct(blendedIm)
cv2.imwrite('../data/Output5.jpg',result[pyramidLevel])


True